# Guard Gallivant

In [2]:
import re

with open("input.txt") as f:
    content = f.read()

def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))

def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])

def get_guard_position(matrix):
    guard_pattern = re.compile(r"(?:<|\^|>|v)")
    for y, line in enumerate(matrix):
        if match := guard_pattern.search("".join(line)):
            position = match.group(0)
            x = line.index(position)
            return x, y, position
    return -1, -1, "x" # went for cup of coffee

def get_next_step(matrix, x, y, position):
    if position == "^":
        if y == 0:
            return y-1, x,"x"
        return x, y-1, matrix[y-1][x]
    
    elif position == ">":
        if x>= len(matrix[y]):
            print(f"len(matrix[y])={len(matrix[y])}, x={x}, y={y}")
            return x+1, y, "x"
        return x+1, y, matrix[y][x+1]
        
    elif position == "<":
        if x == 0:
            return x-1, y, "x"
        return x-1, y, matrix[y][x-1]
        
    elif position == "v":
        if y >= len(matrix)-1:
            return x, y+1, "x"
        return x, y+1, matrix[y+1][x]

def get_new_direction(current):
    directions = ["^", ">", "v", "<", "^"]
    return directions[directions.index(current)+1]

def move_forward(matrix, safety=6000, debug=False):
    while True:
        safety -= 1
        x, y, pos = get_guard_position(matrix)
        nx, ny, nval = get_next_step(matrix, x, y, pos)
        if nval == "x":
            break

        if nval == "#" or nval == "O":
            matrix[y][x] = get_new_direction(pos)
            continue

        matrix[ny][nx] = pos
        matrix[y][x] = "*"

        if debug:
            print(render_matrix(matrix) + "\n\n")

        if not safety:
            # print(render_matrix(matrix) + "\n\n")
            raise RuntimeError("Too much")

    return render_matrix(matrix)
    
matrix = get_matrix(content)
move_forward(matrix).count("*") + 1

5030

Correct: `5030`

In [3]:
def deserialize(value):
    x,y,direction = value.split(",")
    return int(x), int(y), direction


def update_path(matrix, x, y, direction):
    obstacles = ["#", "O"]
    if direction == "^":
        for iy in range(y, -1, -1):
            if matrix[iy][x] in obstacles:
                return
            matrix[iy][x] = "|"

    if direction == ">":
        for ix in range(x, len(matrix[y])):
            if matrix[y][ix] in obstacles:
                return
            matrix[y][ix] = "-"

    if direction == "v":
        for iy in range(y, len(matrix)):
            if matrix[iy][x] in obstacles:
                return
            matrix[iy][x] = "|"        
    
    if direction == "<":
        for ix in range(x, -1, -1):
            if matrix[y][ix] in obstacles:
                return
            matrix[y][ix] = "-"


def visualize(matrix, corners):
    clone = get_matrix(render_matrix(matrix))
    for x, y, pos in [deserialize(corner) for corner in corners]:
        update_path(clone, x,y, pos)
    for x, y, pos in [deserialize(corner) for corner in corners]:
        clone[y][x] = pos
    return clone

In [26]:
# content = """
# ....#.....
# ....+---+#
# ....|...|.
# ..#.|...|.
# ....|..#|.
# ....|...|.
# .#..<---+.
# ........#.
# #.........
# ......#...
# """.strip()

# with open("test.txt") as f:
#     content = f.read()

with open("input.txt") as f:
    content = f.read()

# content = """
# ....#.....
# .#..+---+#
# ..#.|...|.
# ....|...|.
# ....|..#|.
# ....|...|.
# ..<..---+.
# .#......#.
# #.........
# ......#...
# """.strip()

def get_corner(matrix, x, y, direction):
    obstacles = ["#", "O"]
    if direction == "^":
        for iy in range(y, -1, -1):
            if matrix[iy][x] in obstacles:
                return x, iy+1, ">"

    if direction == ">":
        for ix in range(x, len(matrix[y])):
            if matrix[y][ix] in obstacles:
                return ix-1, y, "v"

    if direction == "v":
        for iy in range(y, len(matrix)):
            if matrix[iy][x] in obstacles:
                return x, iy-1, "<"

    if direction == "<":
        for ix in range(x, -1, -1):
            if matrix[y][ix] in obstacles:
                return ix+1, y, "^"

    return -1, -1, "x"

def serialise(x,y,pos):
    return f"{x},{y},{pos}"

def get_loop(matrix):
    clone = get_matrix(render_matrix(matrix))
    corners = [[],[]]
    index = 0
    out_corners = []
    x, y, direction = get_guard_position(matrix)
    nx, ny, nval = get_next_step(clone, x, y, direction)
    if nval == "x":
        return nx, ny, False
    clone[ny][nx] = "O"
    cx, cy = x, y
    # print(render_matrix(clone))
    for i in range(20000):
        cx, cy, direction =  get_corner(clone, cx, cy, direction)

        # print(cx, cy, direction)

        if direction == "x":
            return nx, ny, False, corners[0]
        corner = serialise(cx, cy, direction)
        if corner in corners[index]:
            # print(corners)
            # return nx, ny, True
            index +=1
        
        if index > 1:
            # print(corners)
            if ":".join(corners[0]).endswith(":".join(corners[1])):
                
                return nx, ny, True, corners[0]
            else:
                # print(corners)
                return nx, ny, False, corners[0]

        corners[index].append(corner)
        
    return nx, ny, False, corners[0]


def get_placement(matrix):
    safety = 6000
    placement = []
    sx, sy, _ = get_guard_position(matrix)
    # restricted_x = range(sx, sx+1)
    # restricted_y = range(sy-1, sy)
    while True:
        safety -= 1
        x, y, direction = get_guard_position(matrix)
        nx, ny, next_area = get_next_step(matrix, x, y, direction)
        if next_area == "x":
            break

        if next_area == "#":
            matrix[y][x] = get_new_direction(direction)
            continue

        px, py, is_closed, corners = get_loop(matrix)
        if is_closed:
            # visual = get_matrix(render_matrix(matrix))
            # visual[py][px] = "O"
            # print("\n🔥\n" + render_matrix(visual) + "\n\n")
            placement.append((px, py, corners))

        matrix[ny][nx] = direction
        matrix[y][x] = "*"

        # print(render_matrix(matrix) + "\n\n")

        if not safety:
            raise RuntimeError("Too much")

        # print("-------")
        # print(render_matrix(matrix))
        # print("-------\n\n")
    # print(render_matrix(matrix) + "\n\n")
    return placement


variants = ""

matrix = get_matrix(content)
placements = get_placement(matrix)

count = 0
for x, y, corners in placements:
    count += 1
    matrix = get_matrix(content)
    matrix[y][x] = "O"
    matrix = visualize(matrix, corners)
    variants += f"Loop {count}\n" + render_matrix(matrix) + "\n\n"
    
with open("out.txt", "w") as f:
    f.write(variants)

# final = get_matrix(content)
# for x, y, corners in placements:
#     final[y][x] = "O"
#     visualize(get_matrix(content), corners)
# print(render_matrix(final))

print(f"Count: {len(set([f'{value[0]},{value[1]}' for value in placements]))}")

Count: 1998


# Incorrect

- `1156` - That's not the right answer; your answer is too low
- `1883` - That's not the right answer; your answer is too low.

- `1997` - That's not the right answer.
- `1998` - That's not the right answer.
- `2138` - That's not the right answer; your answer is too high.

In [149]:
# def get_obstacle_position(matrix, x, y, direction):
#     obstacles = ["#", "O"]
#     if direction == "^":
#         for iy in range(y, -1, -1):
#             if matrix[iy][x] in obstacles:
#                 return x, iy

#     if direction == ">":
#         for ix in range(x, len(matrix[y])):
#             if matrix[y][ix] in obstacles:
#                 return ix, y

#     if direction == "v":
#         for iy in range(y, len(matrix)):
#             if matrix[iy][x] in obstacles:
#                 return x, iy

#     if direction == "<":
#         for ix in range(x, -1, -1):
#             if matrix[y][ix] in obstacles:
#                 return ix, y

#     return -1, -1